In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
import matplotlib.pyplot as plt  
import seaborn as sns
%matplotlib inline

In [2]:
df=pd.read_csv("dataset.csv")
print("Data Imported")
df.head()

Data Imported


,Unnamed: 0,submissionId,submissionDate,sem,notif_result,notif_date,studentType,gpa,gre_v,gre_q,gre_w,institution,major,degree
0,0,64506,01-12-2010,F10,Accepted,11/26/2009,A,3.60,154,155,0.0,University College London (UCL) https://www.u...,Public Policy,Master's
1,1,64507,01-12-2010,F10,Accepted,01-11-2010,A,3.60,154,155,0.0,London School of Economics and Political Scien...,Public Administration,Master's
2,2,64902,1/19/2010,F10,Accepted,12-09-2009,A,3.75,167,161,0.0,Cornell University https://www.cornell.edu/,Public Administration,Master's
3,3,64989,1/20/2010,F10,Accepted,1/16/2010,I,3.56,156,161,4.5,University of Southern California (USC) https:...,Computer Science,Master's
4,4,65339,1/25/2010,F10,Accepted,1/25/2010,I,4.00,165,166,4.5,Stanford University https://www.stanford.edu/,Mechanical Engineering,Master's


In [3]:
df=df.dropna(subset=['gpa'])

In [4]:
df=df.drop(["Unnamed: 0","submissionId","submissionDate","notif_date"],axis=1)
df.head()

,sem,notif_result,studentType,gpa,gre_v,gre_q,gre_w,institution,major,degree
0,F10,Accepted,A,3.60,154,155,0.0,University College London (UCL) https://www.u...,Public Policy,Master's
1,F10,Accepted,A,3.60,154,155,0.0,London School of Economics and Political Scien...,Public Administration,Master's
2,F10,Accepted,A,3.75,167,161,0.0,Cornell University https://www.cornell.edu/,Public Administration,Master's
3,F10,Accepted,I,3.56,156,161,4.5,University of Southern California (USC) https:...,Computer Science,Master's
4,F10,Accepted,I,4.00,165,166,4.5,Stanford University https://www.stanford.edu/,Mechanical Engineering,Master's


> Id and Date has no effect on college selection so its better to drop those columns

In [5]:
features=df.columns
for feature in features:
  print("No. of unique values :",df[feature].nunique(),",  ",df[feature].dtype," for ",feature)

No. of unique values : 23 ,   object  for  sem
No. of unique values : 1 ,   object  for  notif_result
No. of unique values : 4 ,   object  for  studentType
No. of unique values : 298 ,   float64  for  gpa
No. of unique values : 37 ,   int64  for  gre_v
No. of unique values : 40 ,   int64  for  gre_q
No. of unique values : 27 ,   float64  for  gre_w
No. of unique values : 259 ,   object  for  institution
No. of unique values : 86 ,   object  for  major
No. of unique values : 1 ,   object  for  degree


> Checking for number of unique values in each column

In [6]:
df=df.drop(["notif_result","degree","studentType"],axis=1)#these col have very few unique values

> These columns have very few uniques values and doesnt contribute to college selection

In [7]:
df.columns

Index(['sem', 'gpa', 'gre_v', 'gre_q', 'gre_w', 'institution', 'major'], dtype='object')

In [8]:
df['major'] = df['major'].astype('str') 

> Converted major object type column to string type

In [9]:
# Label Encoding

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le3 = preprocessing.LabelEncoder()
df['sem'] = le.fit_transform(df['sem'])
df['major'] = le3.fit_transform(df['major'])

df.head()

,sem,gpa,gre_v,gre_q,gre_w,institution,major
0,1,3.60,154,155,0.0,University College London (UCL) https://www.u...,78
1,1,3.60,154,155,0.0,London School of Economics and Political Scien...,77
2,1,3.75,167,161,0.0,Cornell University https://www.cornell.edu/,77
3,1,3.56,156,161,4.5,University of Southern California (USC) https:...,20
4,1,4.00,165,166,4.5,Stanford University https://www.stanford.edu/,57


> Label encoding for object/string type columns 

In [10]:
le2 = preprocessing.LabelEncoder()
df["institution"]= le2.fit_transform(df["institution"])
# p = le2.inverse_transform(y)
df.head()

,sem,gpa,gre_v,gre_q,gre_w,institution,major
0,1,3.60,154,155,0.0,158,78
1,1,3.60,154,155,0.0,81,77
2,1,3.75,167,161,0.0,34,77
3,1,3.56,156,161,4.5,224,20
4,1,4.00,165,166,4.5,145,57


> Seperate label encoding for target column



In [11]:
# splitting
msk=np.random.rand(len(df))<0.85
dftrain=df[msk]
dftest=df[~msk]
#getting target feature from dataset
ytrain=pd.DataFrame(dftrain['institution'])
ytest=pd.DataFrame(dftest['institution'])

#dropping y column from dftrain and dftest
dftrain.drop(["institution"],axis=1,inplace=True)
dftest.drop(["institution"],axis=1,inplace=True)

> Splitting of data into train and test set with 85:15 ratio

## Model Creation

> Importing packages

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score , recall_score, precision_score, f1_score, cohen_kappa_score, roc_auc_score
from sklearn.linear_model import SGDClassifier
import time
from sklearn import metrics

### 1.Decision Tree

In [13]:
acc=[]
timer=[]
criterias=["accuracy_score","recall_score","precision_score","f1_score","cohen_kappa_score","roc_auc_score"]

In [14]:
x=time.time()
model1=DecisionTreeClassifier(max_depth=10)
model1=model1.fit(dftrain,ytrain)

now=(time.time()-x)*1000
print('Time taken = ',now)
timer.append(now)
print('Decision Tree Classifer Created')

Time taken =  190.54698944091797
Decision Tree Classifer Created


In [15]:
ypred = model1.predict(dftest)
ypredtrain=model1.predict(dftrain)
print("Accuracy:",metrics.accuracy_score(ypredtrain, ytrain))



Accuracy: 0.13682941531042797


In [16]:
acc2=[]
tacc2=[]
for criteria in criterias:
  try:
    acc2.append((round(eval(criteria+"(ypredtrain,ytrain)"),2)))
  except:
    if criteria=="roc_auc_score":
      acc2.append(roc_auc_score(ytrain, model1.predict_proba(dftrain), multi_class='ovr'))
    else:
      acc2.append((round(eval(criteria+"(ypredtrain,ytrain,average='micro')"),2)))
      
acc.append(acc2)

> Finding all metrics

### 2. Random Forest

In [17]:
x=time.time()

model3=RandomForestClassifier(criterion= 'entropy', max_depth= 20, max_features='log2', n_estimators= 500)
model3=model3.fit(dftrain,ytrain.values.ravel())

now=(time.time()-x)*1000
print('Time taken = ',now)
timer.append(now)
print('random Forest Classifer Created')

Time taken =  67416.19277000427
random Forest Classifer Created


In [18]:
ypred = model3.predict(dftest)
ypredtrain=model3.predict(dftrain)
print("Accuracy:",metrics.accuracy_score(ypredtrain, ytrain))



Accuracy: 0.8150583593621569


In [19]:
acc2=[]
tacc2=[]
for criteria in criterias:
  try:
    acc2.append((round(eval(criteria+"(ypredtrain,ytrain)"),2)))
  except:
    if criteria=="roc_auc_score":
      acc2.append(roc_auc_score(ytrain, model3.predict_proba(dftrain), multi_class='ovr'))
    else:
      acc2.append((round(eval(criteria+"(ypredtrain,ytrain,average='micro')"),2)))
acc.append(acc2)

### 3. Logistic Regression

In [20]:
x=time.time()

model4=LogisticRegression()
model4=model4.fit(dftrain,ytrain.values.ravel())

now=(time.time()-x)*1000
print('Time taken = ',now)
timer.append(now)
print('Logistic Regression model Created')

Time taken =  22559.741020202637
Logistic Regression model Created


C:\Users\clint\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [21]:
ypred = model4.predict(dftest)
ypredtrain=model4.predict(dftrain)
print("Accuracy:",metrics.accuracy_score(ypredtrain, ytrain))



Accuracy: 0.05019453120718943


In [22]:
acc2=[]
tacc2=[]
for criteria in criterias:
  try:
    acc2.append((round(eval(criteria+"(ypredtrain,ytrain)"),2)))
  except:
    if criteria=="roc_auc_score":
      acc2.append(roc_auc_score(ytrain, model4.predict_proba(dftrain), multi_class='ovr'))
    else:
      acc2.append((round(eval(criteria+"(ypredtrain,ytrain,average='micro')"),2)))
acc.append(acc2)

### 4. K-Nearest Neighbor

In [23]:

from sklearn.model_selection import GridSearchCV
# For knowing best k value
params = {'n_neighbors':[2,3,4,5,6,7,8,9]}

knn = KNeighborsClassifier()

model = GridSearchCV(knn, params, cv=5)
model.fit(dftrain,ytrain.values.ravel())

C:\Users\clint\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9]})

> FInd best value of k using grid search cv

In [24]:
model.best_params_

{'n_neighbors': 9}

In [25]:
x=time.time()

model5=KNeighborsClassifier(8)
model5=model5.fit(dftrain,ytrain.values.ravel())

now=(time.time()-x)*1000
print('Time taken = ',now)
timer.append(now)
print('K_nearest Neighbor model Created')

Time taken =  49.00002479553223
K_nearest Neighbor model Created


In [26]:
ypred = model5.predict(dftest)
ypredtrain=model5.predict(dftrain)
print("Accuracy:",metrics.accuracy_score(ypredtrain, ytrain))


Accuracy: 0.2203956381171571


In [27]:
acc2=[]
tacc2=[]
for criteria in criterias:
  try:
    acc2.append((round(eval(criteria+"(ypredtrain,ytrain)"),2)))
  except:
    if criteria=="roc_auc_score":
      acc2.append(roc_auc_score(ytrain, model5.predict_proba(dftrain), multi_class='ovr'))
    else:
      acc2.append((round(eval(criteria+"(ypredtrain,ytrain,average='micro')"),2)))
acc.append(acc2)

### 5. SVM

In [28]:
x=time.time()

model6=SVC()
model6=model6.fit(dftrain,ytrain.values.ravel())

now=(time.time()-x)*1000
print('Time taken = ',now)
timer.append(now)
print('Support Vector Machine model Created')

Time taken =  13397.722005844116
Support Vector Machine model Created


In [29]:
ypred = model6.predict(dftest)
ypredtrain=model6.predict(dftrain)
print("Accuracy:",metrics.accuracy_score(ypredtrain, ytrain))


Accuracy: 0.04482437393829799


In [30]:
acc2=[]
tacc2=[]
for criteria in criterias:
  try:
    acc2.append((round(eval(criteria+"(ypredtrain,ytrain)"),2)))
  except:
    if criteria=="roc_auc_score":
      # acc2.append(roc_auc_score(ytrain, model6.predict_proba(dftrain), multi_class='ovr'))
      acc2.append("NA")
    else:
      acc2.append((round(eval(criteria+"(ypredtrain,ytrain,average='micro')"),2)))
acc.append(acc2)

## Results

In [31]:
acc=pd.DataFrame(acc)
acc2=acc.rename(columns = {0:criterias[0],1:criterias[1],2:criterias[2],3:criterias[3],4:criterias[4],5:criterias[5]}, inplace = False)
acc2

,accuracy_score,recall_score,precision_score,f1_score,cohen_kappa_score,roc_auc_score
0,0.14,0.14,0.14,0.14,0.12,0.924401
1,0.82,0.82,0.82,0.82,0.81,0.999719
2,0.05,0.05,0.05,0.05,0.02,0.724101
3,0.22,0.22,0.22,0.22,0.21,0.989431
4,0.04,0.04,0.04,0.04,0.00,0.530000


> Metrics of all models

In [32]:
model_names=["decision tree","random forest","logistic regression","K-Nearest Neighbor","Support Vector Machine"]
comparison={'Modelling Algo':model_names,'Timing(in ms)':timer}
acc_frame=pd.DataFrame(comparison)
acc_frame

,Modelling Algo,Timing(in ms)
0,decision tree,190.546989
1,random forest,67416.192770
2,logistic regression,22559.741020
3,K-Nearest Neighbor,49.000025
4,Support Vector Machine,13397.722006


In [33]:
res=pd.concat([acc_frame,acc2],axis=1)
res

,Modelling Algo,Timing(in ms),accuracy_score,recall_score,precision_score,f1_score,cohen_kappa_score,roc_auc_score
0,decision tree,190.546989,0.14,0.14,0.14,0.14,0.12,0.924401
1,random forest,67416.192770,0.82,0.82,0.82,0.82,0.81,0.999719
2,logistic regression,22559.741020,0.05,0.05,0.05,0.05,0.02,0.724101
3,K-Nearest Neighbor,49.000025,0.22,0.22,0.22,0.22,0.21,0.989431
4,Support Vector Machine,13397.722006,0.04,0.04,0.04,0.04,0.00,0.530000


In [34]:
#to get top 10 results from bestmodel i.e. Random Forest on test data
probs = model3.predict_proba(dftest)
best_n = np.argsort(probs, axis=1)[:,-10:]
pred=[]
for i in best_n:
  p2 = le2.inverse_transform(i)
  p2=p2[::-1]
  pred.append(p2)
pd.DataFrame(pred).head()

,0,1,2,3,4,5,6,7,8,9
0,Columbia University https://www.columbia.edu/,University of Florida (UFL) https://www.ufl.edu/,University of South Florida (USF) https://www....,American University https://www.american.edu/,University of Iowa https://uiowa.edu/,Brandeis University https://www.brandeis.edu/,State University of New York at New Paltz (SUN...,Vermont College of Fine Arts https://vcfa.edu/,Stanford University https://www.stanford.edu/,University of Vermont https://www.uvm.edu/
1,University of Idaho https://www.uidaho.edu/,Carnegie Mellon University (CMU) https://www.c...,New York University (NYU) https://www.nyu.edu/,University of Edinburgh https://www.ed.ac.uk/,Columbia University https://www.columbia.edu/,Colorado State University https://www.colostat...,Cornell University https://www.cornell.edu/,Vermont College of Fine Arts https://vcfa.edu/,Harvard University https://www.harvard.edu/,Purdue University https://www.purdue.edu/
2,Southern Connecticut State University https://...,University of Idaho https://www.uidaho.edu/,Columbia University https://www.columbia.edu/,University of South Florida (USF) https://www....,Georgia Institute of Technology (Georgia Tech)...,University of Oxford https://www.ox.ac.uk/,Purdue University https://www.purdue.edu/,University of Suffolk https://www.uos.ac.uk/,Massachusetts Institute of Technology (MIT) ht...,University of Edinburgh https://www.ed.ac.uk/
3,Seton Hall University https://www.shu.edu/,Duquesne University https://www.duq.edu/,Cornell University https://www.cornell.edu/,University of Virginia (UVA) https://www.virgi...,University of Mississippi https://olemiss.edu/,University of South Florida (USF) https://www....,Georgia State University (GSU) https://www.gsu...,McGill University https://www.mcgill.ca/,Stanford University https://www.stanford.edu/,University of New Hampshire https://www.unh.edu/
4,Georgetown University https://www.georgetown.edu/,University of Vermont https://www.uvm.edu/,University of Rhode Island https://www.uri.edu/,Boston College (BC) https://mba.bc.edu/,University of Connecticut (UConn) https://ucon...,University of Idaho https://www.uidaho.edu/,Brandeis University https://www.brandeis.edu/,Vanderbilt University https://www.vanderbilt.edu/,American University https://www.american.edu/,University of Florida (UFL) https://www.ufl.edu/


> Top 10 college list for all test data

In [ ]:
import pickle

with open('le.pickle', 'wb') as handle:
    pickle.dump(le, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('le2.pickle', 'wb') as handle:
    pickle.dump(le2, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('le3.pickle', 'wb') as handle:
    pickle.dump(le3, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('model11.pickle', 'wb') as handle:
    pickle.dump(model3, handle, protocol=pickle.DEFAULT_PROTOCOL)
    

with open('le2.pickle', 'wb') as handle:
    pickle.dump(le2, handle, protocol=pickle.DEFAULT_PROTOCOL)